# [Clustering4Ever](https://github.com/Clustering4Ever/Clustering4Ever) on [SparkNotebook](http://spark-notebook.io/) by [LIPN](https://lipn.univ-paris13.fr/) [A3](https://lipn.univ-paris13.fr/accueil/equipe/a3/) team

# Scala Gradient Ascent

In [ ]:
import smile.plot._
import scala.io.Source
import org.clustering4ever.math.distances.scalar.Euclidean
import org.clustering4ever.util.ScalaImplicits._
import org.clustering4ever.scala.clusterizables.EasyClusterizableExt
import org.clustering4ever.scala.vectorizables.RealVector
import org.clustering4ever.scala.clustering.meanshift.GradientAscent
import org.clustering4ever.scala.kernels.KernelNature
import scala.collection.{immutable, mutable}
import org.clustering4ever.scala.kernels._

import smile.plot._
import scala.io.Source
import org.clustering4ever.math.distances.scalar.Euclidean
import org.clustering4ever.util.ScalaImplicits._
import org.clustering4ever.scala.clusterizables.EasyClusterizableExt
import org.clustering4ever.scala.vectorizables.RealVector
import org.clustering4ever.scala.clustering.meanshift.GradientAscent
import org.clustering4ever.scala.kernels.KernelNature
import scala.collection.{immutable, mutable}
import org.clustering4ever.scala.kernels._


## Download dataset Aggregation

In [ ]:
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
:sh wget -P /tmp/ http://www.clustering4ever.org/Datasets/Aggregation/labels

--2018-11-15 21:31:24--  http://www.clustering4ever.org/Datasets/Aggregation/aggregation.csv
Resolving www.clustering4ever.org (www.clustering4ever.org)... 62.210.16.62
Connecting to www.clustering4ever.org (www.clustering4ever.org)|62.210.16.62|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8063 (7.9K) [text/csv]
Saving to: ‘/tmp/aggregation.csv.1’

     0K .......                                               100% 6.78M=0.001s

2018-11-15 21:31:24 (6.78 MB/s) - ‘/tmp/aggregation.csv.1’ saved [8063/8063]

:sh: Scheme missing.
--2018-11-15 21:31:24--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2018-11-15 21:31:24--  http://www.clustering4ever.org/Datasets/Aggregation/labels
Reusing existing connection to www.clustering4ever.org:80.
HTTP request sent, awaiting response... 200 OK
Length: 1576 (1.5K)
Saving to: ‘/tmp/labels.1’

     0K .                                                    

## Import data as ParArray[ArrayBuffer[Double]]

In [ ]:
val path = "/tmp/aggregation.csv"
val data = Source.fromFile(path).getLines.toSeq.map( x => mutable.ArrayBuffer(x.split(",").map(_.toDouble):_*)).toParArray.zipWithIndex
    .map{ case (v, id) => new EasyClusterizableExt(id, new RealVector(v)) }
val labelsPath = "/tmp/labels"

path: String = /tmp/aggregation.csv
data: scala.collection.parallel.mutable.ParArray[org.clustering4ever.scala.clusterizables.EasyClusterizableExt[Int,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ParArray(EasyClusterizableExt(0,RealVector(ArrayBuffer(15.55, 28.65)),None,Map()), EasyClusterizableExt(1,RealVector(ArrayBuffer(14.9, 27.55)),None,Map()), EasyClusterizableExt(2,RealVector(ArrayBuffer(14.45, 28.35)),None,Map()), EasyClusterizableExt(3,RealVector(ArrayBuffer(14.15, 28.8)),None,Map()), EasyClusterizableExt(4,RealVector(ArrayBuffer(13.75, 28.05)),None,Map()), EasyClusterizableExt(5,RealVector(ArrayBuffer(13.35, 28.45)),None,Map()), EasyClusterizableExt(6,RealVector(ArrayBuffer(13.0, 29.15)),None,Map()), EasyClusterizableExt(7,RealV...

## Parameters 

In [ ]:
val classicEuclidean = new Euclidean[mutable.ArrayBuffer[Double]](false)
val k = 40
val k2 = 1
val epsilon = 0.00001
val maxNumberIter = 50

classicEuclidean: org.clustering4ever.math.distances.scalar.Euclidean[scala.collection.mutable.ArrayBuffer[Double]] = Euclidean distance without root applied
k: Int = 40
k2: Int = 1
epsilon: Double = 1.0E-5
maxNumberIter: Int = 50


In [ ]:
val knnRealKernel = new KernelArgsKnnReal(k = 50, metric = classicEuclidean)
val knnEuclideanKernel = new KernelArgsEuclideanKnn(k = 50, metric = classicEuclidean)
val gaussianArgs = new KernelArgsGaussian(bandwidth = 0.75, metric = classicEuclidean)
val flatArgs = new KernelArgsFlat(bandwidth = 0.75, metric = classicEuclidean, lambda = 1D)
val sigmoidArgs = new KernelArgsSigmoid(a = 1D, b = 3D)

knnRealKernel: org.clustering4ever.scala.kernels.KernelArgsKnnReal[scala.collection.mutable.ArrayBuffer,org.clustering4ever.math.distances.scalar.Euclidean] = KernelArgsKnnReal(50,Euclidean distance without root applied)
knnEuclideanKernel: org.clustering4ever.scala.kernels.KernelArgsEuclideanKnn[scala.collection.mutable.ArrayBuffer,org.clustering4ever.math.distances.scalar.Euclidean] = KernelArgsEuclideanKnn(50,Euclidean distance without root applied)
gaussianArgs: org.clustering4ever.scala.kernels.KernelArgsGaussian[scala.collection.mutable.ArrayBuffer,org.clustering4ever.math.distances.scalar.Euclidean] = KernelArgsGaussian(0.75,Euclidean distance without root applied)
flatArgs: org.clustering4ever.scala.kernels.KernelArgsFlat[scala.collection.mutable.ArrayBuffer,org.clustering4ever....

In [ ]:
val gaussian = new KernelGaussian(gaussianArgs)
val flat = new KernelFlat(flatArgs)
val sigmoid = new KernelSigmoid[mutable.ArrayBuffer](sigmoidArgs)
val knnReal = new KnnKernelReal(knnRealKernel)
val knnEuc = new KnnKernelEuclidean(knnEuclideanKernel)

gaussian: org.clustering4ever.scala.kernels.KernelGaussian[scala.collection.mutable.ArrayBuffer,org.clustering4ever.math.distances.scalar.Euclidean] = KernelGaussian(KernelArgsGaussian(0.75,Euclidean distance without root applied))
flat: org.clustering4ever.scala.kernels.KernelFlat[scala.collection.mutable.ArrayBuffer,org.clustering4ever.math.distances.scalar.Euclidean] = KernelFlat(KernelArgsFlat(0.75,Euclidean distance without root applied,1.0))
sigmoid: org.clustering4ever.scala.kernels.KernelSigmoid[scala.collection.mutable.ArrayBuffer] = KernelSigmoid(KernelArgsSigmoid(1.0,3.0))
knnReal: org.clustering4ever.scala.kernels.KnnKernelReal[scala.collection.mutable.ArrayBuffer,org.clustering4ever.math.distances.scalar.Euclidean,org.clustering4ever.scala.kernels.KernelArgsKnnReal] = KnnKe...

## Run the algorithm

In [ ]:
val t1 = System.currentTimeMillis
val convergedDS =  GradientAscent.run(data, epsilon, maxNumberIter, sigmoid, classicEuclidean, Int.MaxValue)
val t2 = System.currentTimeMillis

(t2 - t1) / 1000D

t1: Long = 1542314103830
convergedDS: scala.collection.GenSeq[org.clustering4ever.scala.clusterizables.EasyClusterizableExt[Int,scala.collection.mutable.ArrayBuffer[Double],scala.collection.mutable.ArrayBuffer[Double]]] = ParArray(EasyClusterizableExt(0,RealVector(ArrayBuffer(15.55, 28.65)),None,Map(2147483647 -> ArrayBuffer(19.56681472081219, 14.171763959390866))), EasyClusterizableExt(1,RealVector(ArrayBuffer(14.9, 27.55)),None,Map(2147483647 -> ArrayBuffer(19.56681472081219, 14.171763959390866))), EasyClusterizableExt(2,RealVector(ArrayBuffer(14.45, 28.35)),None,Map(2147483647 -> ArrayBuffer(19.56681472081219, 14.171763959390866))), EasyClusterizableExt(3,RealVector(ArrayBuffer(14.15, 28.8)),None,Map(2147483647 -> ArrayBuffer(19.56681472081219, 14.171763959390866))), EasyClusterizabl...

0.187



# Plot original and converged datsasets

In [ ]:
plot(convergedDS.map(_.vector.toArray).toArray)
// plot(convergedDS.map(_.altVectors.get(Gradient_Ascent).toArray).toArray)
plot(convergedDS.map(_.altVectors(Int.MaxValue).toArray).toArray)
// plot(convergedDS.map(_.altVectors(1).toArray).toArray)

res35: smile.plot.Window = Window(javax.swing.JFrame[frame4,460,37,1000x1000,invalid,layout=java.awt.BorderLayout,title=Smile Plot 10,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,5,25,990x970,invalid,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777673,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,0x0,invalid,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=9,maximumSize=,minimumSize=,preferredSize=])


Window(javax.swing.JFrame[frame4,460,37,1000x1000,invalid,layout=java.awt.BorderLayout,title=Smile Plot 10,resizable,normal,defaultCloseOperation=DISPOSE_ON_CLOSE,rootPane=javax.swing.JRootPane[,5,25,990x970,invalid,layout=javax.swing.JRootPane$RootLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=16777673,maximumSize=,minimumSize=,preferredSize=],rootPaneCheckingEnabled=true],smile.plot.PlotCanvas[,0,0,0x0,invalid,layout=java.awt.BorderLayout,alignmentX=0.0,alignmentY=0.0,border=,flags=9,maximumSize=,minimumSize=,preferredSize=])